[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/08-DWT/MDWT_compression.ipynb)

<a href="https://colab.research.google.com/github/vicente-gonzalez-ruiz/MCDWT-experimentos/blob/master/src/MDWT_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RD in the MDWT (Motion Discrete Wavelet Transform) domain

Optimal RD should consider the slopes of all the points of all the images of the sequence.

Unfinished.

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import math
import numpy as np
from scipy import signal
import cv2
import os
import pywt
import pylab
!ln -sf ~/quantization/deadzone_quantizer.py .
import deadzone_quantizer as deadzone
!ln -sf ~/MRVC/src/image_3.py .
import image_3 as frame
!ln -sf ~/MRVC/src/YCoCg.py .
import YCoCg
!ln -sf ~/MRVC/src/DWT.py .
!ln -sf ~/MRVC/src/L_DWT.py .
!ln -sf ~/MRVC/src/H_DWT.py .
import DWT
!ln -sf ~/quantization/information.py .
!ln -sf ~/quantization/distortion.py .
import distortion

In [ ]:
def q_deq(x, quantization_step):
    k = deadzone.quantize(x, quantization_step)
    y = deadzone.dequantize(k, quantization_step)
    return y, k

In [ ]:
def load_indexes(prefix):
    frame.read(prefix)
    
def write_indexes(prefix):
    frame.write(prefix)

In [ ]:
def show(img, title):
    int = frame.normalize(ing)
    plt.figure(figsize=(10,10))
    plt.title(title, fontsize=20)
    plt.imshow(img)

In [ ]:
def average_energy(x):
    return np.sum(x.astype(np.double)*x.astype(np.double))/(np.size(x))

def MSE(x, y):
    error_signal = x.astype(np.float32) - y
    return average_energy(error_signal)

def RMSE(x, y):
    error_signal = xx.astype(np.float32) - y
    return math.sqrt(MSE(error_signal))

def bytes_per_frame(img):
    frame.write(img, "/tmp/frame", 0)
    length_in_bytes = os.path.getsize("/tmp/frame000.png")
    return length_in_bytes

def bytes_per_grayframe(img):
    cv2.imwrite("/tmp/frame.png", img)
    length_in_bytes = os.path.getsize("/tmp/frame.png")
    return length_in_bytes

In [ ]:
WAVELET = pywt.Wavelet("db5")
#WAVELET = pywt.Wavelet("bior3.5")
N_levels = 3

def DWT_RD_curve(RGB_frame, N_levels):
    n_channels = RGB_frame.shape[2]
    RD_points = []
    for q_step in range(0, 8):
        YUV_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        YUV_decomposition = DWT.analyze(YUV_frame, N_levels=N_levels)
        dequantized_YUV_decomposition = []
        rate = 0
        cAn = YUV_decomposition[0]
        dequantized_cAn, k = q_deq(cAn, 1<<q_step)
        #rate += bytes_per_frame(k.astype(np.uint8))
        rate += frame.write((k + 128).astype(np.uint8), f"/tmp/{q_step}_", 0)
        dequantized_YUV_decomposition.append(dequantized_cAn)
        rest_of_resolutions = YUV_decomposition[1:]
        for resolution in rest_of_resolutions:
            dequantized_resolution = []
            for subband in resolution:
                dequantized_subband, k  = q_deq(subband, 1<<q_step)
                #rate += bytes_per_frame(k.astype(np.uint8))
                rate += frame.write((k + 128).astype(np.uint8), f"/tmp/{q_step}_", 0)
                dequantized_resolution.append(dequantized_subband)
            dequantized_YUV_decomposition.append(tuple(dequantized_resolution))
        reconstructed_YUV_frame = DWT.synthesize(dequantized_YUV_decomposition, N_levels=N_levels)
        reconstructed_RGB_frame = YCoCg.to_RGB(reconstructed_YUV_frame)
        _distortion = distortion.MSE(RGB_frame, reconstructed_RGB_frame)
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={_distortion:>6.1f}")
        RD_points.append((8*rate/RGB_frame.size, _distortion))
    return RD_points

def _DWT_RD_curve(RGB_frame, N_levels):
    n_channels = RGB_frame.shape[2]
    RD_points = []
    for q_step in range(0, 8):
        YCoCg_frame = YCoCg.from_RGB(RGB_frame.astype(np.int16))
        YCoCg_decomposition = DWT.analyze(YCoCg_frame, N_levels=N_levels)
        dequantized_YCoCg_decomposition = []
        rate = 0
        for channel in range(n_channels):
            # In a channel there is a decomposition
            decomposition = YCoCg_decomposition[channel]
            cAn = decomposition[0]
            dequantized_cAn, k = q_deq(cAn, 1<<q_step)
            k = k.astype(np.uint8)
            dequantized_decomposition = [dequantized_cAn]
            rate += bytes_per_frame(k)
            rest_of_resolutions = decomposition[1:]
            for resolution in rest_of_resolutions:
                # In a resolution there is/are one/three subbands
                dequantized_resolution = []
                for subband in resolution:
                    dequantized_subband, k = q_deq(subband, 1<<q_step)
                    k = k.astype(np.uint8)
                    rate += bytes_per_grayframe(k)
                    dequantized_resolution.append(dequantized_subband)
                dequantized_decomposition.append(tuple(dequantized_resolution))
            dequantized_YCoCg_decomposition.append(dequantized_decomposition)
        reconstructed_YCoCg_frame = DWT.synthesize(dequantized_YCoCg_decomposition, N_levels=N_levels)
        reconstructed_RGB_frame = YCoCg.to_RGB(reconstructed_YCoCg_frame)
        distortion = MSE(RGB_frame, reconstructed_RGB_frame)
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
        RD_points.append((rate, distortion))
    return RD_points

def _DWT_RD_curve(RGB_frame, N_levels=N_levels):
    n_channels = RGB_frame.shape[0]
    RD_points = []
    for q_step in range(0, 8):
        YCoCg_frame = YCoCg.from_RGB(RGB_frame)
        YCoCg_decomposition = DWT.analyze(YCoCg_frame, N_levels=N_levels)
        dequantized_YCoCg_decomposition = []
        rate = 0
        for channel in range(n_channels):
            # In a channel there is a decomposition
            decomposition = YCoCg_decomposition[channel]
            cAn = decomposition[0]
            k, dequantized_cAn = q_deq(cAn, 1<<q_step)
            dequantized_decomposition = [dequantized_cAn]
            rate += bytes_per_frame(k)
            rest_of_resolutions = decomposition[1:]
            for resolution in rest_of_resolutions:
                # In a resolution there is/are one/three subbands
                dequantized_resolution = []
                for subband in resolution:
                    k, dequantized_subband = q_deq(subband, 1<<q_step)
                    rate += bytes_per_frame(k)
                    dequantized_resolution.append(dequantized_subband)
                dequantized_decomposition.append(tuple(dequantized_resolution))
            dequantized_YCoCg_decomposition.append(dequantized_decomposition)
        reconstructed_YCoCg_frame = DWT.synthesize(dequantized_YCoCg_decomposition)
        reconstructed_RGB_frame = YCoCg.to_RGB(reconstructed_YCoCg_frame)
        distortion = MSE(RGB_frame, reconstructed_RGB_frame)
        print(f"q_step={1<<q_step:>3}, rate={rate:>7} bytes, distortion={distortion:>6.1f}")
        RD_points.append((rate, distortion))
    return RD_points

In [ ]:
RD_curves = []
for frame_number in range(5):
    fn = f"/home/vruiz/MRVC/sequences/stockholm/"
    print(fn)
    RGB_frame = frame.read(fn, frame_number)
    RD_points = DWT_RD_curve(RGB_frame, N_levels=3)
    RD_curves.append(RD_points)

In [ ]:
pylab.figure(dpi=150)
for frame_number in range(5):
    pylab.plot(*zip(*RD_curves[frame_number]), marker="x", label=f'{frame_number:03d}')
pylab.title("RD Curves of Consecutive Frames")
pylab.xlabel("Bytes/Frame")
pylab.ylabel("MSE")
plt.legend(loc='upper right')
plt.yscale('log')
#plt.xscale('log')
pylab.show()